In [5]:
from csv import writer
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [6]:
url='http://www.cgv.co.kr/movies/detail-view/?midx=87433'

In [7]:
option = webdriver.ChromeOptions()
option.headless = True
option.add_argument('window-size=1920x1080')

In [11]:
def get_movie_reviews(urs, page_num=10):
    wd = webdriver.Chrome(options=option)
    wd.get(url)
    writer_list = []
    review_list = []
    date_list = []

    for page_no in range(1, page_num + 1):
        try:
            page_ul = wd.find_element(By.ID, 'paging_point') 
            page_a = page_ul.find_element(By.LINK_TEXT, str(page_no))# 페이지 숫자
            page_a.click()
            time.sleep(1)

            writers = wd.find_elements(By.CLASS_NAME, 'writer-name')
            writer_list += [writer.text for writer in writers]

            reviews = wd.find_elements(By.CLASS_NAME, 'box-comment')
            review_list += [review.text for review in reviews]

            dates = wd.find_elements(By.CLASS_NAME, 'day')
            date_list += [date.text for date in dates]

            if page_no % 10 == 0:
                next_button = page_ul.find_element(
                    By.CLASS_NAME, 'btn-paging.next')
                next_button.click()
                time.sleep(1)
        except NoSuchElementException:
            break

    movie_review_df = pd.DataFrame({'Writer': writer_list,
                                    'Review': review_list,
                                    'Date':date_list})
    return movie_review_df
movie_review_df = get_movie_reviews(url, 100)
movie_review_df

movie_review_df.to_csv('C:\\sqlite\\mysql\\code\\AI\\selenium_webdriver\\selenium_webdriver.csv', encoding='utf-8-sig', index=False)
            